In [10]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json
import os

In [11]:
!mkdir img

In [8]:
!ls

all_models_url.jsonl  get-content.ipynb  get-url.ipynb	pic


In [12]:
with open('all_models_specs.jsonl') as fopen:
    specs = [line.strip().strip('"') for line in fopen]

In [287]:
# specs

In [13]:
len(specs)

1589

In [14]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36',
           'Referer':'https://www.wapcar.my/'}

In [15]:
# Define a mapping from Unicode characters to strings
unicode_to_string = {
    '\ue605': 'no',
    '\ue604': 'yes',
    '\ue605 / \ue605': 'no / no',
    '\ue604 / \ue604': 'yes / yes',
    '\ue604 / \ue605': 'yes / no',
    '\ue605 / \ue604': 'no / yes',
    ' \ue603 \ue603 \ue603 \ue603 \ue603': '5 stars',
    ' \ue603 \ue603 \ue603 \ue603 \ue601': '4 stars',
    ' \ue603 \ue603 \ue603 \ue601 \ue601': '3 stars',
    ' \ue603 \ue603 \ue601 \ue601 \ue601': '2 stars',
    ' \ue603 \ue601 \ue601 \ue601 \ue601': '1 star',
    ' \ue601 \ue601 \ue601 \ue601 \ue601': '0 star',
}

# Example usage
input_char = '\ue605 \ue605 \ue605'  # This would be your input character

# Convert the Unicode character to the corresponding string
output_string = unicode_to_string.get(input_char, input_char)

print(output_string)  # This will print 'no' or 'yes' based on the input character


  


In [7]:
# url = "https://www.wapcar.my/cars/proton/saga/1.3-standard-mt-2018/specs"
# r = requests.get(url,headers = headers)
# soup = BeautifulSoup(r.content, "html.parser")

# data_specs = []
# title = {'model' : soup.find('div', {'class': 'car-name'}).find('a').text}
# count=0
# data_specs.append(title)
# for i in soup.find('div', {'class': 'content'}).find('table', {'class': 'value-table table-row'}).find_all('tr'):
#     attr = {}
#     try:
#         if len(i.get('class'))>2:
#             attr[' '.join(i.get('class'))] = unicode_to_string.get(i.find(['a', 'span']).text, i.find(['a', 'span']).text)
#             data_specs.append(attr)
#             count+=1
#     except Exception as e:
#         print(e)

# data_specs

In [16]:
# url = "https://www.wapcar.my/cars/proton/saga/1.3-standard-mt-2018/images"
# r = requests.get(url,headers = headers)
# soup = BeautifulSoup(r.content, "html.parser")
# img_data = []
# count=0
# for i in soup.find_all('script', {'type':'application/ld+json'}):
    
#     img_dict = {}
#     data = json.loads(i.get_text())
#     if data['@type'] == 'ImageObject':
#         count+=1
#         url = data['contentUrl']
    
#         img_file = os.path.join('pic', f'{count}.jpg')
#         with open(img_file, 'wb') as fopen:
#             r = requests.get(url)
#             fopen.write(r.content)
            
#         img_dict['img_url'] = data['contentUrl']
#         img_dict['local_image'] = img_file
#         img_dict['specs'] = data_specs
#         img_data.append(img_dict)
        

In [285]:
# img_data

In [13]:
# count=0

In [17]:
def crawl(x):
    try:
        url_specs = "https://www.wapcar.my" + x + '/specs'
        r = requests.get(url_specs,headers = headers)
        soup = BeautifulSoup(r.content, "html.parser")
    
        data_specs = []
        
        title = {'model' : soup.find('div', {'class': 'car-name'}).find('a').text}
        data_specs.append(title)
        
        for i in soup.find('div', {'class': 'content'}).find('table', {'class': 'value-table table-row'}).find_all('tr'):
            attr = {}
            
            if len(i.get('class'))>2:
                attr[' '.join(i.get('class'))] = unicode_to_string.get(i.find(['a', 'span']).text, i.find(['a', 'span']).text)
                data_specs.append(attr)

                
        url_images = "https://www.wapcar.my" + x + '/images'  
        r = requests.get(url_images,headers = headers)
        soup = BeautifulSoup(r.content, "html.parser")
        
        img_data = []
        count=0
        for i in soup.find_all('script', {'type':'application/ld+json'}):
            count+=1
            img_dict = {}
            data = json.loads(i.get_text())
            if data['@type'] == 'ImageObject':
                count+=1
                url = data['contentUrl']
            
                img_file = os.path.join('img', f"{data_specs[0]['model']}_{count}.jpg")
                with open(img_file, 'wb') as fopen:
                    r = requests.get(url)
                    fopen.write(r.content)
                    
                img_dict['img_url'] = data['contentUrl']
                img_dict['local_image'] = img_file
                img_dict['specs'] = data_specs
                img_data.append(img_dict)
                
        return img_data
         
    except Exception as e: 
        print(e)
        return 

In [18]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from tqdm import tqdm

max_workers = 10
urls = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(crawl, x) for x in specs]
    
    for future in tqdm(futures, total=len(specs)):
        result = future.result()
        if result:
            for x in result:
                with open('model_specs.jsonl', 'a') as f:
                    json.dump(x, f)
                    f.write('\n')

  2%|██▌                                                                                                         | 38/1589 [00:13<08:26,  3.06it/s]

'NoneType' object has no attribute 'find_all'


 11%|███████████▍                                                                                               | 170/1589 [01:05<08:12,  2.88it/s]

'NoneType' object has no attribute 'find'


 11%|████████████                                                                                               | 180/1589 [01:08<07:52,  2.98it/s]

...

 69%|█████████████████████████████████████████████████████████████████████████▏                                | 1097/1589 [08:26<05:53,  1.39it/s]

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'


 71%|██████████████████████████████████████████████████████████████████████████▊                               | 1121/1589 [08:34<03:29,  2.23it/s]

[Errno 2] No such file or directory: 'img/2018 Nissan X-Gear 1.6L (A/T)_7.jpg'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1589/1589 [12:48<00:00,  2.07it/s]


In [22]:
with open('model_specs.jsonl') as fopen:
    specs = [line.strip().strip('"') for line in fopen]

In [24]:
specs[700]

'{"img_url": "https://images.wapcar.my/file/d4a3800ed4da40a3b369c1be075245a6.jpg", "local_image": "img/2019 Alfa Romeo Quadrifoglo Stelvio Quadrifoglio_25.jpg", "specs": [{"model": "2019 Alfa Romeo Quadrifoglo Stelvio Quadrifoglio"}, {"attribute-col Price-block 1-price-col col-background": "TBC"}, {"attribute-col Costs-block 2-insurance-col col-background": "TBC"}, {"attribute-col Costs-block 2-road_tax-col col-background": "TBC"}, {"attribute-col Costs-block 2-monthly_payment-col col-background": "TBC"}, {"attribute-col Overview-block overview-brand_name-col col-background": "Alfa Romeo"}, {"attribute-col Overview-block overview-body_type-col col-background": "Sedan"}, {"attribute-col Overview-block overview-segment-col col-background": "B-Segment"}, {"attribute-col Overview-block overview-fuel_type-col col-background": "-"}, {"attribute-col Overview-block overview-model_name-col col-background": "Alfa Romeo Quadrifoglio"}, {"attribute-col Overview-block overview-year-col col-backgrou